<a href="https://colab.research.google.com/github/Joshuajee/AI-ML-PROJECTS/blob/master/Catch%20the%20LLM%20Text%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification Articles written by LLMs

## Setup

In [ ]:
!pip install tensorflow==2.15 tensorflow-hub==0.15

In [ ]:
import numpy as np
import pandas as pd
import requests
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.framework import config as tf_config
tf_config.enable_op_determinism()
tf.random.set_seed(100)

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## Download the data from Github

The dataset is available on https://github.com/Joshuajee/AI-ML-PROJECTS/master/. The following code downloads the IMDB dataset to your machine (or the colab runtime):

In [ ]:
def get_model_data_from_github(model):
  file_path = "https://raw.githubusercontent.com/Joshuajee/AI-ML-PROJECTS/master/data/llms/"
  reponse = requests.get(file_path + model)
  if reponse.status_code == 200:
    with open(model, "wb") as file:
      file.write(reponse.content)
  else:
    raise Exception("Error: downloading", model, reponse.status_code)

In [ ]:
# Download the model data from github this is faster
get_model_data_from_github("Meta-Llama-3-8B-Instruct.Q4_0.gguf.csv")
get_model_data_from_github("Phi-3-mini-4k-instruct.Q4_0.gguf.csv")
get_model_data_from_github("Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf.csv")
get_model_data_from_github("orca-mini-3b-gguf2-q4_0.gguf.csv")

## Explore the data

Exploring the data to better understand the response gotten from the LLMs.

From the exploration Meta-Llama-3-8B-Instruct tend to use this "\*\*" character to represent heading, this will help to easily recognise the response gotten from Meta-Llama-3-8B-Instruct but because "\*\*" is not a word they will be removed from our data.

In [ ]:
# Read and explore Meta-Llama-3-8B-Instruct data
meta = pd.read_csv("Meta-Llama-3-8B-Instruct.Q4_0.gguf.csv")
meta.head(10)

In [ ]:
# Read and explore Phi-3-mini-4k-instruct data
phi = pd.read_csv("Phi-3-mini-4k-instruct.Q4_0.gguf.csv")
phi.head(10)

In [ ]:
# Read and explore Nous-Hermes-2-Mistral-7B-DPO.Q4_0 data
nous = pd.read_csv("Nous-Hermes-2-Mistral-7B-DPO.Q4_0.gguf.csv")
nous.head(10)

In [ ]:
# Read and explore orca-mini-3b-gguf2-q4_0.gguf data
orca = pd.read_csv("orca-mini-3b-gguf2-q4_0.gguf.csv")
orca.head(10)

Get the total count of the DataFrames in other to balance the classes

In [ ]:
print(phi.count())
print(meta.count())
print(nous.count())
print(orca.count())

In [ ]:
complete_df = pd.concat([meta, phi, nous, orca], ignore_index=True)
complete_df

In [ ]:
# Remove trailing and leading whitespaces on the dataframe columns
complete_df.columns = complete_df.columns.str.strip()

### Calculating and plotting the average words per model

In [ ]:
# Function to count words
complete_df['word_count'] = complete_df['Response'].apply(lambda x: len(x.split()))

# Calculate average words per Model
avg_words_per_model = complete_df.groupby('Model', as_index=False)['word_count'].mean()

avg_words_per_model

In [ ]:
plt.figure(figsize=(20, 8))
sns.barplot(x='Model', y='word_count', data=avg_words_per_model, palette='viridis')

# Customization
plt.xlabel('Model')
plt.ylabel('Average Word Count')
plt.title('Average Word Count per Model')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

## Preprocessing



Very few text cleaning is needed here because of the text embedding, but I will remove the "**" character that is in most of Meta-Llama-3-8B-Instruct responses.

In [ ]:
complete_df['Response'] = complete_df['Response'].str.replace('**', ' ').str.strip()
complete_df

### Encoding
Computers don’t understand text, so we need to convert our labels (the model) into numbers. The label encoder is a good library for this task.


In [ ]:
complete_df['Features'] = complete_df['Response']
encoder = LabelEncoder()
complete_df['Labels'] = encoder.fit_transform(complete_df['Model'])
complete_df

In [ ]:
# Create a new DataFrame with only the Features and Labels columns
feature_df = complete_df[['Features', 'Labels']]
feature_df

## Splitting Data
Spliting into training, validation, and testing sets.

In [ ]:
train_data, temp_data = train_test_split(feature_df, test_size=0.4, stratify=feature_df['Labels'], random_state=42)
train_data

In [ ]:
# Further split the temp_data into validation and test sets
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['Labels'], random_state=42)
val_data

In [ ]:
test_data

### Converting to Tensors
In order to work with Tensorflow we have to convert our train, validation and test datasets into Tensors.

In [ ]:
# Converting to Tensors
train_features = tf.convert_to_tensor(train_data['Features'].values, dtype=tf.string) # Convert 'Features' to tf.string tensor
train_labels = tf.convert_to_tensor(to_categorical(train_data['Labels'].values, num_classes=4), dtype=tf.int64)   # Convert 'Labels' to tf.int64 tensor

val_features = tf.convert_to_tensor(val_data['Features'].values, dtype=tf.string) # Convert 'Features' to tf.string tensor
val_labels = tf.convert_to_tensor(to_categorical(val_data['Labels'].values, num_classes=4), dtype=tf.int64)   # Convert 'Labels' to tf.int64 tensor

test_features = tf.convert_to_tensor(test_data['Features'].values, dtype=tf.string) # Convert 'Features' to tf.string tensor
test_labels = tf.convert_to_tensor(to_categorical(test_data['Labels'].values, num_classes=4), dtype=tf.int64)   # Convert 'Labels' to tf.int64 tensor

## Build the model

The neural network is created by stacking layers—this requires three main architectural decisions:

* How to represent the text?
* How many layers to use in the model?
* How many *hidden units* to use for each layer?

In this project, the input data consists of sentences. The labels to predict are either 0, 1, 2, 3.

To represent the text, the sentences will be converted into embeddings vectors.

Pre-trained text embedding will be used as the first layer, which will have two advantages:
*   we don't have to worry about text preprocessing,
*   we can benefit from transfer learning.

For this example I will use the following model.

1. [Universal Sentence Encoder v4](https://tfhub.dev/google/universal-sentence-encoder/4).

2. [google/nnlm-en-dim50-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2).

3. [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2)

The layers are stacked sequentially to build the classifier:

1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. We are using the three models above to splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: `(num_featuress, embedding_dimension)`.
2. This fixed-length output vector is piped through a fully-connected (`Dense`) layer with 16 hidden units.
3. The last layer is densely connected with a four output node. This outputs logits: the log-odds of the true class, according to the model.

Let's now build the full model:

In [ ]:
def build_model(embedding_model):
    model = tf.keras.Sequential()
    model.add(hub.KerasLayer(embedding_model, input_shape=[], dtype=tf.string, trainable=True))
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
universal_sen_encoder = build_model("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
nnlm_50_dim_norm = build_model("https://tfhub.dev/google/nnlm-en-dim50-with-normalization/2")

In [ ]:
nnlm_128_dim_norm = build_model("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2")

### Hidden units

The above model has two intermediate or "hidden" layers, between the input and output. The number of outputs (units, nodes, or neurons) is the dimension of the representational space for the layer. In other words, the amount of freedom the network is allowed when learning an internal representation.


### Loss function and optimizer

A model needs a loss function and an optimizer for training. Since this is a categorical classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the `categorical_crossentropy` loss function.

## Train the models

Train the model for 15 epochs in mini-batches of 100 samples. This is 360 iterations over all samples in the `x_train` and `y_train` tensors.

In [ ]:
total_epochs = 15
def train_model(model, train_features, train_labels, val_features, val_labels):
  return model.fit(train_features, train_labels, epochs=total_epochs, batch_size=100, validation_data=(val_features, val_labels), verbose=1)

In [ ]:
universal_sen_encoder_history = train_model(universal_sen_encoder, train_features, train_labels, val_features, val_labels)

In [ ]:
nnlm_50_dim_norm_history = train_model(nnlm_50_dim_norm, train_features, train_labels, val_features, val_labels)

In [ ]:
nnlm_128_dim_norm_history = train_model(nnlm_128_dim_norm, train_features, train_labels, val_features, val_labels)

## Evaluate the models

And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, lower values are better), and accuracy.

In [ ]:
def evaluate_model(model, test_features, test_labels):
  results = model.evaluate(test_features, test_labels)
  print(f"test loss: {results[0]}, test acc: {results[1]}")
  return results

In [ ]:
universal_sen_encoder_results = evaluate_model(universal_sen_encoder, test_features, test_labels)

In [ ]:
nnlm_50_dim_norm_results = evaluate_model(nnlm_50_dim_norm, test_features, test_labels)

In [ ]:
nnlm_128_dim_norm_results = evaluate_model(nnlm_128_dim_norm, test_features, test_labels)

Model predict, this is assuming that we feed a brand new data to our trained system

In [ ]:
def plot_confusion_matrix(model, name):
    class_names = ['Meta-Llama-3-8B-Instruct', 'Phi-3-mini-4k-instruct', 'Nous-Hermes-2-Mistral-7B-DPO', 'orca-mini-3b-gguf2-q4_0']
    y_pred = model.predict(test_features)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(test_labels, axis=1)
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(name + ' Confusion Matrix')
    plt.show()
    return cm

In [ ]:
universal_sen_encoder_cm = plot_confusion_matrix(universal_sen_encoder, "Universal Sentence Encoder")

In [ ]:
nnlm_50_dim_norm_cm = plot_confusion_matrix(nnlm_50_dim_norm, "NNLM-EN-50-DIM")

In [ ]:
nnlm_128_dim_norm_cm = plot_confusion_matrix(nnlm_128_dim_norm, "NNLM-EN-128-DIM")

## Create a graph of accuracy and loss over time

In [ ]:
# Getting the metric array from the history for the three models
(universal_sen_train_acc, universal_sen_train_loss, universal_sen_val_acc, universal_sen_val_loss) = get_model_metric(universal_sen_encoder_history)
(nnlm_50_dim_norm_train_acc, nnlm_50_dim_norm_train_loss, nnlm_50_dim_norm_val_acc, nnlm_50_dim_norm_val_loss) = get_model_metric(nnlm_50_dim_norm_history)
(nnlm_128_dim_norm_train_acc, nnlm_128_dim_norm_train_loss, nnlm_128_dim_norm_val_acc, nnlm_128_dim_norm_val_loss) = get_model_metric(nnlm_128_dim_norm_history)

In [ ]:
epochs = np.arange(1, total_epochs + 1)

def get_model_metric(history):
  history_dict = history.history
  history_dict.keys()
  train_acc = history_dict['accuracy']
  train_loss = history_dict['loss']
  val_acc = history_dict['val_accuracy']
  val_loss = history_dict['val_loss']
  return train_acc, train_loss, val_acc, val_loss

def plot_training_metric(plot_for = "Accuracy"):

  (universal_sen_train_acc, universal_sen_train_loss, universal_sen_val_acc, universal_sen_val_loss) = get_model_metric(universal_sen_encoder_history)
  (nnlm_50_dim_norm_train_acc, nnlm_50_dim_norm_train_loss, nnlm_50_dim_norm_val_acc, nnlm_50_dim_norm_val_loss) = get_model_metric(nnlm_50_dim_norm_history)
  (nnlm_128_dim_norm_train_acc, nnlm_128_dim_norm_train_loss, nnlm_128_dim_norm_val_acc, nnlm_128_dim_norm_val_loss) = get_model_metric(nnlm_128_dim_norm_history)

  if plot_for == "Accuracy":
    universal_sen_train = universal_sen_train_acc
    nnlm_50_dim_norm_train = nnlm_50_dim_norm_train_acc
    nnlm_128_dim_norm_train = nnlm_128_dim_norm_train_acc

    universal_sen_encoder_val = universal_sen_val_acc
    nnlm_50_dim_norm_val = nnlm_50_dim_norm_val_acc
    nnlm_128_dim_norm_val = nnlm_128_dim_norm_val_acc
  elif plot_for == "Loss":
    universal_sen_train = universal_sen_train_loss
    nnlm_50_dim_norm_train = nnlm_50_dim_norm_train_loss
    nnlm_128_dim_norm_train = nnlm_128_dim_norm_train_loss

    universal_sen_encoder_val = universal_sen_val_loss
    nnlm_50_dim_norm_val = nnlm_50_dim_norm_val_loss
    nnlm_128_dim_norm_val = nnlm_128_dim_norm_val_loss
  else:
    return




  plt.clf()
  sns.set_theme(style="whitegrid")
  plt.figure(figsize=(16, 8))

  # Plot for universal-sentence-encoder
  plt.plot(epochs, universal_sen_train, marker='o', linestyle='-', color='green', label='universal-sentence-encoder Training ' + plot_for)
  plt.plot(epochs, universal_sen_encoder_val,   marker='o', linestyle='--', color='green', label='universal-sentence-encoder Validation ' + plot_for)

  # Plot for nnlm-en-dim50
  plt.plot(epochs, nnlm_50_dim_norm_train, marker='o', linestyle='-', color='blue', label='nlm-en-dim50 Training ' + plot_for)
  plt.plot(epochs, nnlm_50_dim_norm_val,   marker='o', linestyle='--', color='blue', label='nlm-en-dim50 Validation ' + plot_for)

  # Plot for nnlm-en-dim128
  plt.plot(epochs, nnlm_128_dim_norm_train, marker='o', linestyle='-', color='red', label='nlm-en-dim128 Training ' + plot_for)
  plt.plot(epochs, nnlm_128_dim_norm_val,   marker='o', linestyle='--', color='red', label='nlm-en-dim128 Validation ' + plot_for)


  # Labeling and legend
  plt.title("Training and Validation " + plot_for)
  plt.xlabel("Epochs")
  plt.ylabel(plot_for)
  plt.xticks(epochs)

  # Labeling and legend
  plt.title("Training and Validation " + plot_for)
  plt.xlabel("Epochs")
  plt.ylabel(plot_for)
  plt.xticks(epochs)  # The epoch will be on the x-axis

  plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)

  plt.tight_layout()
  plt.show()

### Training and Validation accuracy

In [ ]:
plot_training_metric("Accuracy")

In [ ]:
plot_training_metric("Loss")